# Week 16: Reinforcement Learning for Trading

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **RL Fundamentals**: States, Actions, Rewards
- **Q-Learning**: Value-based learning
- **Policy Gradient**: Direct policy optimization
- **Trading Environment**: Formulating trading as RL

---

## Why RL for Trading?

Trading is inherently sequential decision-making:
- **State**: Current market conditions
- **Action**: Buy, sell, hold
- **Reward**: Profit/loss
- **Goal**: Maximize cumulative reward

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 16: Reinforcement Learning")

---

## Part 1: RL Framework

### Markov Decision Process (MDP)

- **State space S**: Market conditions
- **Action space A**: Trading decisions
- **Transition P(s'|s,a)**: Market dynamics
- **Reward R(s,a,s')**: Trading returns

### Objective

Maximize expected discounted return:

$$G_t = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$$

### 🤔 Simple Explanation

RL is learning by doing. The agent takes actions, sees results, and adjusts strategy. Like a trader learning from P&L.

In [ ]:
class TradingEnvironment:
    """Simple trading environment"""
    
    def __init__(self, prices, transaction_cost=0.001):
        self.prices = prices
        self.transaction_cost = transaction_cost
        self.n_steps = len(prices)
        self.reset()
    
    def reset(self):
        self.current_step = 0
        self.position = 0  # -1, 0, or 1
        self.portfolio_value = 1.0
        return self._get_state()
    
    def _get_state(self):
        # Simple state: past 5 returns + position
        if self.current_step < 5:
            returns = np.zeros(5)
        else:
            returns = np.diff(np.log(self.prices[self.current_step-5:self.current_step+1]))
        return np.append(returns, self.position)
    
    def step(self, action):
        """Action: 0=sell, 1=hold, 2=buy"""
        new_position = action - 1  # Convert to -1, 0, 1
        
        # Calculate return
        price_return = (self.prices[self.current_step + 1] - 
                       self.prices[self.current_step]) / self.prices[self.current_step]
        
        # Position return
        pnl = self.position * price_return
        
        # Transaction cost if position changed
        if new_position != self.position:
            pnl -= self.transaction_cost
        
        self.portfolio_value *= (1 + pnl)
        self.position = new_position
        self.current_step += 1
        
        done = self.current_step >= self.n_steps - 1
        
        return self._get_state(), pnl, done

# Test environment
prices = 100 * np.cumprod(1 + np.random.randn(100) * 0.02)
env = TradingEnvironment(prices)
state = env.reset()
print(f"Initial state: {state}")

# Random action
next_state, reward, done = env.step(2)  # Buy
print(f"After BUY: reward={reward:.4f}, done={done}")

---

## Part 2: Q-Learning

### Value Function

$$Q(s, a) = E[G_t | S_t = s, A_t = a]$$

### Bellman Equation

$$Q(s, a) = R + \gamma \max_{a'} Q(s', a')$$

### Q-Learning Update

$$Q(s, a) \leftarrow Q(s, a) + \alpha [R + \gamma \max_{a'} Q(s', a') - Q(s, a)]$$

In [ ]:
class QLearningAgent:
    def __init__(self, state_bins=10, n_actions=3, lr=0.1, gamma=0.95, epsilon=0.1):
        self.state_bins = state_bins
        self.n_actions = n_actions
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        
        # Q-table (simplified: discretize state space)
        # State: 5 returns + position → discretize to bins
        self.q_table = {}
    
    def _discretize_state(self, state):
        # Simple discretization
        bins = np.linspace(-0.1, 0.1, self.state_bins)
        discrete = tuple(np.digitize(state[:-1], bins).tolist() + [int(state[-1] + 1)])
        return discrete
    
    def get_action(self, state):
        state_d = self._discretize_state(state)
        
        # Epsilon-greedy
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        
        # Greedy
        if state_d not in self.q_table:
            self.q_table[state_d] = np.zeros(self.n_actions)
        return np.argmax(self.q_table[state_d])
    
    def update(self, state, action, reward, next_state, done):
        state_d = self._discretize_state(state)
        next_state_d = self._discretize_state(next_state)
        
        if state_d not in self.q_table:
            self.q_table[state_d] = np.zeros(self.n_actions)
        if next_state_d not in self.q_table:
            self.q_table[next_state_d] = np.zeros(self.n_actions)
        
        # Q-learning update
        target = reward
        if not done:
            target += self.gamma * np.max(self.q_table[next_state_d])
        
        self.q_table[state_d][action] += self.lr * (target - self.q_table[state_d][action])

In [ ]:
# Train Q-learning agent
def train_agent(env, agent, episodes=100):
    rewards_history = []
    
    for ep in range(episodes):
        state = env.reset()
        total_reward = 0
        
        while True:
            action = agent.get_action(state)
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state, done)
            
            total_reward += reward
            state = next_state
            
            if done:
                break
        
        rewards_history.append(total_reward)
    
    return rewards_history

# Generate training data
prices_train = 100 * np.cumprod(1 + np.random.randn(500) * 0.02)
env = TradingEnvironment(prices_train, transaction_cost=0.001)
agent = QLearningAgent(epsilon=0.2)

rewards = train_agent(env, agent, episodes=50)

# Plot learning curve
plt.figure(figsize=(10, 4))
plt.plot(pd.Series(rewards).rolling(10).mean())
plt.xlabel('Episode')
plt.ylabel('Total Reward (10-ep MA)')
plt.title('Q-Learning Training')
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 3: Deep Q-Network (DQN)

### Neural Network for Q-Function

Instead of a table, use a neural network:

$$Q(s, a; \theta) \approx Q^*(s, a)$$

### Key Improvements

1. **Experience Replay**: Store transitions, sample randomly
2. **Target Network**: Stable targets for training
3. **Continuous states**: Handle high-dimensional states

In [ ]:
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from collections import deque
    
    class DQN(nn.Module):
        def __init__(self, state_dim, n_actions):
            super().__init__()
            self.network = nn.Sequential(
                nn.Linear(state_dim, 64),
                nn.ReLU(),
                nn.Linear(64, 32),
                nn.ReLU(),
                nn.Linear(32, n_actions)
            )
        
        def forward(self, x):
            return self.network(x)
    
    print("DQN Architecture:")
    model = DQN(state_dim=6, n_actions=3)
    print(model)
    
except ImportError:
    print("⚠️ PyTorch not installed")

---

## Part 4: Policy Gradient

### Direct Policy Optimization

Instead of learning value, learn policy directly:

$$\pi_\theta(a|s) = P(A_t = a | S_t = s; \theta)$$

### REINFORCE Algorithm

$$\nabla_\theta J(\theta) = E[\sum_t \nabla_\theta \log \pi_\theta(a_t|s_t) G_t]$$

### 🤔 Simple Explanation

Policy gradient directly learns "what to do" instead of "how good is this state." Good outcomes increase the probability of the actions that led to them.

---

## Interview Questions

### Conceptual
1. Why is trading a good fit for RL?
2. What's the difference between Q-learning and policy gradient?
3. What is the exploration-exploitation trade-off?

### Technical
1. How do you define the reward function for trading?
2. What are the challenges of applying RL to finance?
3. How does experience replay help DQN?

### Finance-Specific
1. What state representation would you use for stocks?
2. How do you handle transaction costs in RL?
3. Can RL learn risk management?

---

## Key Takeaways

| Method | Learns | Best For |
|--------|--------|----------|
| Q-Learning | Value function | Discrete actions |
| DQN | Q via neural net | Complex states |
| Policy Gradient | Policy directly | Continuous actions |